<a href="https://colab.research.google.com/github/pangeab-blip/EvGeo-Exercises/blob/main/ALBA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ALBA mini-protocol (single cell, Colab/Jupyter) — versione con grafici F_comp e ΔT
import numpy as np
import math
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, Dropdown

R_earth = 6371e3
A_earth = 4*math.pi*R_earth**2
A_earth_Mkm2 = A_earth/1e12
sigma = 5.670374419e-8

def hemi_band_area_Mkm2(phi1_deg, phi2_deg):
    phi1 = math.radians(phi1_deg); phi2 = math.radians(phi2_deg)
    return (2*math.pi*R_earth**2 * (math.sin(phi2) - math.sin(phi1)))/1e12

bands = [(0,10),(10,35),(35,65),(65,90)]
A_hemi = np.array([hemi_band_area_Mkm2(b[0], b[1]) for b in bands])

def alba_miniprot(
    A_land_tot=150.0, share_NH=0.60,
    nh_s0=0.10, nh_s1=0.30, nh_s2=0.40, nh_s3=0.20,
    sh_s0=0.10, sh_s1=0.30, sh_s2=0.40, sh_s3=0.20,
    aL_0_10=0.15, aL_10_35=0.18, aL_35_65=0.25, aL_65_90=0.40,
    aO_0_10=0.06, aO_10_35=0.07, aO_35_65=0.08, aO_65_90=0.10,
    S0=1361.0, conv_mode="Planck-only", lambda_fixed=0.60
):
    A_land_tot = max(140.0, min(160.0, A_land_tot))
    A_land_N = A_land_tot * max(0.0, min(1.0, share_NH))
    A_land_S = A_land_tot - A_land_N

    nh_vec = np.array([nh_s0, nh_s1, nh_s2, nh_s3], float)
    sh_vec = np.array([sh_s0, sh_s1, sh_s2, sh_s3], float)
    nh_vec = nh_vec/nh_vec.sum() if nh_vec.sum()>0 else np.full(4,0.25)
    sh_vec = sh_vec/sh_vec.sum() if sh_vec.sum()>0 else np.full(4,0.25)

    L_N = A_land_N * nh_vec
    L_S = A_land_S * sh_vec

    fL_N = np.clip(L_N / A_hemi, 0.0, 1.0)
    fL_S = np.clip(L_S / A_hemi, 0.0, 1.0)

    aL = np.clip(np.array([aL_0_10, aL_10_35, aL_35_65, aL_65_90]), 0.0, 1.0)
    aO = np.clip(np.array([aO_0_10, aO_10_35, aO_35_65, aO_65_90]), 0.0, 1.0)

    aS_N = fL_N*aL + (1.0 - fL_N)*aO
    aS_S = fL_S*aL + (1.0 - fL_S)*aO

    RWT_N = float(np.sum(aS_N * (A_hemi / A_earth_Mkm2)))
    RWT_S = float(np.sum(aS_S * (A_hemi / A_earth_Mkm2)))

    dRWT = abs(RWT_N - RWT_S)
    RWT_high = max(RWT_N, RWT_S)
    RWT_low  = min(RWT_N, RWT_S)
    RWT_comp = RWT_high + RWT_low + dRWT   # = 2 * RWT_high

    F_comp = - dRWT * (S0/4.0)  # W m^-2

    if conv_mode == "Planck-only":
        Te = ((1.0 - RWT_comp) * S0 / (4.0*sigma))**0.25
        lambda_P = 4.0*sigma*Te**3
        dT = F_comp / lambda_P
    else:
        dT = lambda_fixed * F_comp

    print("=== ALBA mini-protocol ===")
    print(f"RWT_N = {RWT_N:.5f} | RWT_S = {RWT_S:.5f} | ΔRWT = {dRWT:.5f}")
    print(f"RWT_comp = {RWT_comp:.5f} | F_comp = {F_comp:.3f} W m^-2 | ΔT = {dT:.3f} K")
    if conv_mode == "Planck-only":
        print("Conversione ΔF→ΔT: Planck-only (λ_P calcolato da Te).")
    else:
        print(f"Conversione ΔF→ΔT: λ fisso = {lambda_fixed:.2f} K/(W m^-2).")

    # Grafico 1: RWT emisferico
    plt.figure(figsize=(6,4))
    plt.bar(["RWT_N","RWT_S"], [RWT_N, RWT_S])
    plt.ylabel("RWT (adimensionale)")
    plt.title("RWT per emisfero")
    plt.grid(True, axis='y', linestyle=':')
    plt.show()

    # Grafico 2: Albedo di banda per emisfero
    x = np.arange(4); width=0.35
    plt.figure(figsize=(7,4))
    plt.bar(x - width/2, aS_N, width, label='NH')
    plt.bar(x + width/2, aS_S, width, label='SH')
    plt.xticks(x, ["0–10","10–35","35–65","65–90"])
    plt.ylabel("α_surface (banda)")
    plt.title("Albedo superficiale per banda (NH vs SH)")
    plt.legend()
    plt.grid(True, axis='y', linestyle=':')
    plt.show()

    # Grafico 3: Forzante F_comp
    plt.figure(figsize=(6,3.8))
    plt.bar(["F_comp"], [F_comp])
    plt.ylabel("W m$^{-2}$")
    plt.title(f"Forzante compensazione emisferica: F_comp = {F_comp:.3f} W m$^{{-2}}$")
    plt.grid(True, axis='y', linestyle=':')
    plt.show()

    # Grafico 4: ΔT
    plt.figure(figsize=(6,3.8))
    plt.bar(["ΔT"], [dT])
    plt.ylabel("K")
    if conv_mode == "Planck-only":
        plt.title(f"Risposta termica (Planck-only): ΔT = {dT:.3f} K")
    else:
        plt.title(f"Risposta termica (λ fisso): ΔT = {dT:.3f} K")
    plt.grid(True, axis='y', linestyle=':')
    plt.show()

    return {"RWT_N":RWT_N, "RWT_S":RWT_S, "RWT_comp":RWT_comp, "ΔRWT":dRWT,
            "F_comp":F_comp, "ΔT":dT}

interact(
    alba_miniprot,
    A_land_tot=FloatSlider(value=150.0, min=140.0, max=160.0, step=0.5, description="Land tot (Mkm²)"),
    share_NH=FloatSlider(value=0.60, min=0.0, max=1.0, step=0.01, description="NH land share"),
    nh_s0=FloatSlider(value=0.10, min=0.0, max=1.0, step=0.01, description="NH 0–10"),
    nh_s1=FloatSlider(value=0.30, min=0.0, max=1.0, step=0.01, description="NH 10–35"),
    nh_s2=FloatSlider(value=0.40, min=0.0, max=1.0, step=0.01, description="NH 35–65"),
    nh_s3=FloatSlider(value=0.20, min=0.0, max=1.0, step=0.01, description="NH 65–90"),
    sh_s0=FloatSlider(value=0.10, min=0.0, max=1.0, step=0.01, description="SH 0–10"),
    sh_s1=FloatSlider(value=0.30, min=0.0, max=1.0, step=0.01, description="SH 10–35"),
    sh_s2=FloatSlider(value=0.40, min=0.0, max=1.0, step=0.01, description="SH 35–65"),
    sh_s3=FloatSlider(value=0.20, min=0.0, max=1.0, step=0.01, description="SH 65–90"),
    aL_0_10=FloatSlider(value=0.15, min=0.05, max=0.60, step=0.005, description="α_L 0–10"),
    aL_10_35=FloatSlider(value=0.18, min=0.05, max=0.60, step=0.005, description="α_L 10–35"),
    aL_35_65=FloatSlider(value=0.25, min=0.05, max=0.80, step=0.005, description="α_L 35–65"),
    aL_65_90=FloatSlider(value=0.40, min=0.05, max=0.90, step=0.005, description="α_L 65–90"),
    aO_0_10=FloatSlider(value=0.06, min=0.02, max=0.20, step=0.001, description="α_O 0–10"),
    aO_10_35=FloatSlider(value=0.07, min=0.02, max=0.20, step=0.001, description="α_O 10–35"),
    aO_35_65=FloatSlider(value=0.08, min=0.02, max=0.25, step=0.001, description="α_O 35–65"),
    aO_65_90=FloatSlider(value=0.10, min=0.02, max=0.30, step=0.001, description="α_O 65–90"),
    S0=FloatSlider(value=1361.0, min=1300.0, max=1400.0, step=1.0, description="S0 (W m⁻²)"),
    conv_mode=Dropdown(options=["Planck-only","Fixed-λ"], value="Planck-only", description="ΔF→ΔT"),
    lambda_fixed=FloatSlider(value=0.60, min=0.2, max=1.0, step=0.01, description="λ_fixed K/(W m⁻²)")
);

interactive(children=(FloatSlider(value=150.0, description='Land tot (Mkm²)', max=160.0, min=140.0, step=0.5),…